# 04 Att Lstm Model Attention Weights
> ⚠️ This notebook supports the attention-based hybrid strategy  
> (Notebook `08_Hybrid_Attention_Allocation.ipynb`) by generating and exporting 
> attention weights and processed input sequences.

This notebook builds and runs the Attention-LSTM model, then extracts **real attention weights** and aligns them with actual prediction dates for interpretability and integration into hybrid strategy logic.

## 1. Load Raw Data Used to Create Sequences

In [ ]:
import os
import requests
import pandas as pd
import numpy as np
import logging
from time import sleep

In [ ]:
# --- API Configuration ---
API_KEY = os.getenv("EODHD_API_KEY")  # Load API Key securely
TICKER = "GSPC.INDX"
START_DATE = "2015-01-01"
END_DATE = "2025-01-01"
BASE_URL = "https://eodhd.com/api/eod/"

In [ ]:
def fetch_eod_data(ticker, api_key, start_date, end_date, retries=3, delay=5):
    """
    Fetches historical market data from EODHD API with retry logic.

    Parameters:
    - ticker (str): Stock or index ticker symbol
    - api_key (str): API authentication token
    - start_date (str): Start date for data retrieval
    - end_date (str): End date for data retrieval
    - retries (int): Number of retry attempts in case of failure
    - delay (int): Delay between retries (exponential backoff)

    Returns:
    - pd.DataFrame: Market data if successful, else None
    """
    url = f"{BASE_URL}{ticker}?api_token={api_key}&from={start_date}&to={end_date}&fmt=json"
    
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if data:
                    logging.info(f"Successfully retrieved {len(data)} records.")
                    return pd.DataFrame(data)
                else:
                    logging.warning("API returned an empty dataset.")
            else:
                logging.error(f"API request failed with status {response.status_code}: {response.text}")
        except requests.RequestException as e:
            logging.error(f"API request error: {e}")

        sleep(delay * (2 ** attempt))  # Exponential backoff
    return None

In [ ]:
df = fetch_eod_data(TICKER, API_KEY, START_DATE, END_DATE)

if df is not None:
    
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date").reset_index(drop=True)
    
    df.set_index("date", inplace=True)

    numeric_cols = ["open", "high", "low", "close", "adjusted_close", "volume"]
    df[numeric_cols] = df[numeric_cols].astype(float)

    df.ffill(inplace=True)  

    display(df.head())
    

## 2. Load Saved Sequences & Test Index 

In [ ]:
import numpy as np

# Load saved numpy sequences and test dates
X_seq_test = np.load("../data/X_seq_test.npy")
# Assuming test_index.npy contains datetime-formatted index
test_index = pd.to_datetime(np.load("../data/test_index.npy"))

print("Test Sequence Shape:", X_seq_test.shape)
print("Sample Dates:", test_index[:5])

## 3. Rebuild Attention-LSTM Model (Same as Training)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Multiply
from tensorflow.keras.optimizers.legacy import Adam

# Define Attention Layer
def attention_layer(inputs):
    attention_scores = Dense(1, activation='tanh', name="attention_scores")(inputs)
    attention_scores = tf.squeeze(attention_scores, axis=-1)
    attention_weights = tf.nn.softmax(attention_scores, axis=1, name="attention_weights")
    context_vector = Multiply(name="weighted_sum")([inputs, tf.expand_dims(attention_weights, axis=-1)])
    context_vector = tf.reduce_sum(context_vector, axis=1, name="context_vector")
    return context_vector, attention_weights

# Model Function
def get_attention_model(input_shape, lstm_units=80, dropout_rate=0.1, lr=2.6e-5):
    inputs = Input(shape=input_shape, name="input_layer")
    lstm_out = LSTM(lstm_units, return_sequences=True, name="lstm_layer")(inputs)
    lstm_out = Dropout(dropout_rate, name="dropout_layer")(lstm_out)
    context_vector, attention_weights = attention_layer(lstm_out)
    output = Dense(1, activation='sigmoid', name="output_layer")(context_vector)
    model = Model(inputs, output)
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=lr), metrics=['accuracy'])
    return model

# Rebuild Model
input_shape = (X_seq_test.shape[1], X_seq_test.shape[2])
attention_model = get_attention_model(input_shape)
attention_model.summary()

## 4. Load Trained Weights 

In [ ]:
attention_model.save_weights("../models/att_lstm_weights.h5")

## 5. Predict and Extract Attention Weights 

In [ ]:
predictions = attention_model.predict(X_seq_test, verbose=0)

# Access attention weights properly from internal tensor
extract_attention = Model(inputs=attention_model.input,
                          outputs=attention_model.get_layer("dropout_layer").output)

lstm_output = extract_attention.predict(X_seq_test)

# Now reapply attention to this LSTM output
attention_scores = tf.squeeze(Dense(1, activation='tanh')(lstm_output), axis=-1)
attention_weights = tf.nn.softmax(attention_scores, axis=1)

# Collapse across time
attention_mean = attention_weights.numpy().mean(axis=1)

## 6. Align and Save Attention Scores DataFrame 

In [ ]:
# Align index to match predictions/attention size
att_df = pd.DataFrame({
    'predictions': predictions.flatten(),
    'attention_mean': attention_mean
}, index=test_index[-len(predictions):])  # Use last 746 dates

# Save
att_df.to_csv("../data/df_att_with_attention.csv")
att_df.head()
